In [ ]:
import glob
import cupy as cp
import numpy as np
from datetime import datetime
from hist._cmd_line_util import open_input_image
from hist.exact import __calc_info, __check_h_dst, __sort_pixels, __calc_transform, __apply_transform
from utils import clear_line


# 2D and 3D Images
images = glob.glob('../c4l-image-dataset/*/*')
n_imgs = len(images)
for i, filename in enumerate(images):
    string = f'Progress: {i}/{n_imgs}\tCurrent image: {filename[21:]}'
    print(string, end='')

    # Setup for function calls
    timing_data = []
    img_cpu = open_input_image(filename)
    img_gpu = cp.array(img_cpu)

    
    n = img_gpu.size
    h_dst_cpu = __check_h_dst(256, n)
    h_dst_gpu = cp.array(h_dst_cpu, h_dst_cpu.dtype)
    
    
    # GPU timing
    values_gpu = __calc_info(img_gpu, 'va')
    
    data = %timeit -q -o __sort_pixels(values_gpu, img_gpu.shape, None, False, True)
    timing_data.append(data)
    idx_gpu, fails = __sort_pixels(values_gpu, img_gpu.shape, None, False, True)
    del values_gpu
    
    data = %timeit -q -o __calc_transform(h_dst_gpu, img_gpu.dtype, n, idx_gpu)
    timing_data.append(data)
    transform_gpu = __calc_transform(h_dst_gpu, img_gpu.dtype, n, idx_gpu)
    del h_dst_gpu
    
    data = %timeit -q -o __apply_transform(idx_gpu, transform_gpu, img_gpu.shape, None)
    timing_data.append(data)
    
    
    # CPU timing
    values_cpu = __calc_info(img_cpu, 'va')
    
    data = %timeit -q -o __sort_pixels(values_cpu, img_cpu.shape, None, False, True)
    timing_data.append(data)
    idx_cpu, fails = __sort_pixels(values_cpu, img_cpu.shape, None, False, True)
    del values_cpu
    
    data = %timeit -q -o __calc_transform(h_dst_cpu, img_cpu.dtype, n, idx_cpu)
    timing_data.append(data)
    transform_cpu = __calc_transform(h_dst_cpu, img_cpu.dtype, n, idx_cpu)
    del h_dst_cpu
    
    data = %timeit -q -o __apply_transform(idx_cpu, transform_cpu, img_cpu.shape, None)
    timing_data.append(data)
    
    # Save data in file; appends to the end of the file
    with open('int_steps_temp.txt', 'a+') as file:
        file.write('\n\n' + filename + '\n')
        for data in timing_data:
            file.write(str(data) + '\n')
    
    clear_line(len(string)) # Clears the current output

print('Finished!')
        

In [5]:
import os
import os.path
from utils import prefix_to_micro

def convert_temp_to_data():
    if os.path.exists('int_steps_temp.txt'):
        with open('int_steps_temp.txt', 'r') as temp:
            with open('int_steps.csv', 'w+') as file:
                file.write('Image,__sort_pixels_gpu Mean,__sort_pixels_gpu Std. Dev.,__calc_transform_gpu Mean,'
                           '__calc_transform_gpu Std. Dev.,__apply_transform_gpu Mean,__apply_transform_gpu Std. Dev.,'
                           '__sort_pixels_cpu Mean,__sort_pixels_cpu Std. Dev.,__calc_transform_cpu Mean,'
                           '__calc_transform_cpu Std. Dev.,__apply_transform_cpu Mean,__apply_transform_cpu Std. Dev.\n')
                data = temp.read().strip().split('\n\n')
                for img in data:
                    img = img.strip().split('\n')
                    data_list = [img[0].strip('..').strip('/')]
                    # Get time values and convert them to microseconds
                    for i in range(1, 7):
                        vals = img[i].split(' ')
                        data_list.append(float(vals[0])*prefix_to_micro(vals[1]))
                        data_list.append(float(vals[3])*prefix_to_micro(vals[4]))
                    file.write(','.join([str(_) for _ in data_list]) + '\n')

    #os.remove('int_steps_temp.txt')

In [6]:
convert_temp_to_data()